In [1]:
import os
os.environ['OMP_NUM_THREADS']='2'
os.environ['LD_LIBRARY_PATH']=''
os.environ['CUDA_LAUNCH_BLOCKING']='1'

In [2]:
%cd /home/pz281@ad.eng.cam.ac.uk/mnt/PhD/Pro_Down_SR

/home/pz281@ad.eng.cam.ac.uk/mnt/PhD/Pro_Down_SR


In [3]:
from data_generation import *
from scipy.linalg import sqrtm
from downscaling import *
from utils import *

/home/pz281@ad.eng.cam.ac.uk/anaconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Generate Training Data

In [4]:
N_low = 30
N_high = 120
scale = 4

# Code downscaling matrix
H = np.zeros((N_low*N_low, N_high*N_high))

submatrix = np.zeros((N_low,N_high))
for i in range(N_low):
    submatrix[i,scale*i] = 1
    
for j in range(N_low):
    H[N_low*j:N_low*(j+1),N_high*scale*j:N_high*(scale*j+1)] = submatrix
    
for a in [4,8]:
    for b in tqdm(range(1,9)):
        for c in tqdm(range(1,9)):
            w_low = generate_data(N_low,a,b,c)[0]
            w_high = generate_data(N_high,a,b,c)[0]
            residual = w_low-(H@w_high.reshape(N_high*N_high)).reshape(N_low,N_low)
            
            if a==4 and b == 1 and c == 1:
                total_low = w_low.reshape(1,N_low,N_low)
                total_high = w_high.reshape(1,N_high,N_high)
                total_residual = residual.reshape(1,N_low,N_low)
            else:
                total_low = np.concatenate([total_low,w_low.reshape(1,N_low,N_low)],axis=0)
                total_high = np.concatenate([total_high,w_high.reshape(1,N_high,N_high)],axis=0)
                total_residual = np.concatenate([total_residual,residual.reshape(1,N_low,N_low)],axis=0)

100%|██████████| 8/8 [18:21<00:00, 137.74s/it]


In [5]:
with h5py.File("DownBy4_30_120.h5", 'w') as hf:
    hf.create_dataset("high_res",  data=total_high)
    hf.create_dataset("low_res",  data=total_low)
    hf.create_dataset("residual", data=total_residual)

In [7]:
create_forcing_term(20,8,3,3).shape

(400,)

In [4]:
N_low = 30
N_high = 120
scale = 4

# Code downscaling matrix
H = np.zeros((N_low*N_low, N_high*N_high))

submatrix = np.zeros((N_low,N_high))
for i in range(N_low):
    submatrix[i,scale*i] = 1
    
for j in range(N_low):
    H[N_low*j:N_low*(j+1),N_high*scale*j:N_high*(scale*j+1)] = submatrix
    
for a in [4,8]:
    for b in tqdm(range(1,9)):
        for c in tqdm(range(1,9)):
            w_low = generate_data(N_low,a,b,c)[0]
            b_high = create_forcing_term(N_high,a,b,c)
            
            if a==4 and b == 1 and c == 1:
                total_low = w_low.reshape(1,N_low,N_low)
                total_forcing = b_high.reshape(1,N_high**2,1)
            else:
                total_low = np.concatenate([total_low,w_low.reshape(1,N_low,N_low)],axis=0)
                total_forcing = np.concatenate([total_forcing,b_high.reshape(1,N_high**2,1)],axis=0)

100%|██████████| 8/8 [00:02<00:00,  3.60it/s]


In [5]:
with h5py.File("data/30_120_low_forcing.h5", 'w') as hf:
    hf.create_dataset("low_res",  data=total_low)
    hf.create_dataset("forcing", data=total_forcing)